In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from p24asolver import P24ASolver

In [2]:
#events
def sat_radius(t, Y, *args):
    x1 = Y[0]
    x2= Y[2]
    return np.sqrt(x1**2+x2**2)

In [39]:
class Planets(P24ASolver):
    """
    Simulate a satelite swinging around jupiter
    """

    def __init__(self, **kwargs):
        """
        I'm going to use Python's keyword argument mechanism to handle the parameters.

        The kwargs parameter is a dictionary of all the named parameters and their values. We can use
        the built-in 'get' method to store parameter values, which allows us to provide default values
        in cases where no value was given.

        Note that we need to call the __init__ method of P24ASolver with a list of
        variable names. The first string in each variable tuple is the variable name, which you can use to
        refer to the variable, the second is a LaTeX representation that will look nice on a plot.
        """
        super().__init__(
            (('x1', '$x_1$'), ('v1', r'$\dot{x}_1$'),
             ('x2', '$x_2$'), ('v2', r'$\dot{x}_2$'))
        )
        # Now store variables, using defaults, if necessary
        self.JUPR = kwargs.get('JUPR', 1)
        self.JUPW = kwargs.get('JUPW', 1)
        self.JUPM = kwargs.get('JUPM', 1)
        self.SUNM = kwargs.get('SUNM', 1)
        self.G = 10000
        self.SATM = kwargs.get('SATM', 1)
        self.t_off = kwargs.get('t_off',0)
#         self.events.append(radius)
        
        

    def derivatives(self, t, Y):
        x1, v1, x2, v2 = Y  # unpack the values
        xj1, xj2 = self.jup_coords(t) #jup pos
        
        #################### for position of sat#####################
        
        
        ####getting squared radii from sun and jup respectively#####
        R_sun_sq = x1**2+x2**2
        R_jup_sq = (x1-xj1)**2 + (x2-xj2)**2
        #####################################################
        
        ######getting force due to sun and jup respectively##       
        F_sun_mag = (self.G*self.SUNM*self.SATM)/R_sun_sq
        F_sun = (F_sun_mag)*np.array([-x1,-x2])
        
        F_jup_mag = (self.G*self.JUPM*self.SATM)/R_jup_sq
        F_jup = (F_jup_mag)*np.array([xj1-x1,xj2-x2])  
        ###################################################### 
        
        ###############summing those up ##############
        F_tot = F_sun + F_jup
        
        F1, F2 = F_tot
        
        a1 = F1/self.SATM
        
        a2 = F2/self.SATM

        return [v1, a1, v2, a2]

    def orbit(self, R, w, t):
        """
        input: R is the average radius of the planet, t is the time passed, w is the angular speed of the planet
        output: returns the position of the planet at time t given 
        """
        return (R*np.math.cos(w*t), R*np.math.sin(w*t))
    
    def jup_coords(self,t):
        return self.orbit(self.JUPR,self.JUPW,t+self.t_off)

    

In [40]:
rest = (1,0,1,0)
sol1 = Planets(JUPR = 5.2, JUPW = 2*np.pi/12, JUPM = .000955, SUNM = 1, SATM = 2.2268*10**(-31), t_off = 1).solve(rest, (0,.2))

In [ ]:
def closest_approach(v1,v2,ap_dist):
    """
    v1 and v2 are the velocities at launch, ap_dist is the distance you want between jupiter and the satellite
    
    returns t_off to initialize a solar system where the sat and jup get ap_dist apart or gives you the closest occurance if the ap_dist can't be found
    """
    current_t_off = 0
    min_dist = (100000000, 0)
    while current_t_off <= 12:
        times = np.linspace(0,.2,100)
        sol1 = Planets(JUPR = 5.2, JUPW = 2*np.pi/12, JUPM = .000955, SUNM = 1, SATM = 2.2268*10**(-31), t_off = current_t_off).solve(rest, (0,.2))
        for t in times:
            x1 = sol1(t)[0]
            x2 = sol1(t)[1]
            
            xj1 = sol1.jup_coords(t)[0]
            xj2 = sol1.jup_coords(t)[1]
            
            dist = np.math.sqrt((x1-xj1)**2+(x2-xj2)**2)
            if dist <= ap_dist:
                return (dist, current_t_off,t)
            
            if (dist,0) < min_dist:
                min_dist = (dist,current_t_off, t)
        current_t_off += .01
    return min_dist

closest_approach(60,0,0.00233444264)

In [ ]:
sol1 = Planets(JUPR = 5.2, JUPW = 2*np.pi/12, JUPM = .000955, SUNM = 1, SATM = 2.2268*10**(-31), t_off = .29).solve(rest, (0,.2))

sol1(0.05858585858585859)

sol1.jup_coords(0.05858585858585859)